In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
def int_sum(x):
    sum1 = 0
    for i in range(1, x + 1):
        sum1 += i
    return sum1

In [3]:
DATA_PATH = Path.cwd().parent.joinpath('data', 'processed', 'three_factor_model.txt')

In [4]:
df = pd.read_csv(DATA_PATH, delimiter="\t")

In [5]:
df["Price_tb"] = df["Price_tb"] / 100

In [6]:
df.rename(columns={"excess": "mkt_excess"}, inplace=True)

In [7]:
##making SMB
dat = {'datadate': [], 'SMB value': []}
SMB = pd.DataFrame(data=dat)
j = 0
for i in np.unique(df["datadate"]):
    temp = df[df.datadate == i][["chng", "mkvaltq"]]

    ten = np.nanpercentile(temp["mkvaltq"], 10)
    ninety = np.nanpercentile(temp["mkvaltq"], 90)
    small = temp[temp["mkvaltq"] < ten]
    big = temp[temp["mkvaltq"] > ninety]
    small_rets = np.nanmean(small["chng"])
    big_rets = np.nanmean(big["chng"])
    factor = small_rets - big_rets
    SMB.loc[j] = [i, factor]
    j += 1

In [8]:
##making HML
dat = {'datadate': [], 'HML value': []}
HML = pd.DataFrame(data=dat)
j = 0
for i in np.unique(df["datadate"]):
    temp = df[df.datadate == i][["tic", "chng", "ptb"]]

    ten = np.nanpercentile(temp["ptb"], 10)
    ninety = np.nanpercentile(temp["ptb"], 90)
    value = temp[temp["ptb"] < ten]
    growth = temp[temp["ptb"] > ninety]
    val_rets = np.nanmean(value["chng"])
    growth_rets = np.nanmean(growth["chng"])
    factor = val_rets - growth_rets
    HML.loc[j] = [i, factor]
    j += 1

In [9]:
df = df.merge(SMB, how="left", on="datadate")

In [10]:
df = df.merge(HML, how="left", on="datadate")

In [11]:
##pathlib
##make a new data path to the directory that i want
##call path for dunder file (__file__)
from pathlib import Path

data_Path = Path.cwd().parent.joinpath('data', 'processed', 'factor_data')
if not data_Path.exists():
    data_Path.mkdir()

In [12]:
for xxx in np.unique(df["datadate"])[20:]:
    new_df = df[df["datadate"] < xxx]
    last_date = np.unique(new_df["datadate"])[-1]
    factors = pd.DataFrame(
        columns=['Tic', 'Alpha', 'Beta_mkt_Exc', 'Beta_SMB', 'Beta_HML', 'p_MX', 'p_smb', 'p_hml', 'r2'])
    q = -1
    for tic in np.unique(new_df["tic"]):
        if new_df[new_df["tic"] == tic]["datadate"].iloc[-1] != last_date:
            continue
        try:
            q += 1
            temp_df = new_df[new_df["tic"] == tic]
            temp_df = temp_df.reset_index(drop=True)
            temp_df = temp_df.dropna()
            Y = temp_df["chng"] - (temp_df["Price_tb"])
            X = temp_df[["mkt_excess", "SMB value", "HML value"]]
            w = np.ones(temp_df.shape[0])
            denom = int_sum(len(w))
            for i in range(len(w)):
                w[i] += i + 1
            w = w / denom
            model = sm.WLS(Y, X, weights=w).fit()
            alf = 0
            for i in range(len(w)):
                no = 0
                for j in range(len(model.params)):
                    no += X.iloc[i][j] * model.params[j]
                yes = (Y.iloc[i] - no) * w[i]
                alf += yes

            e_i = []
            for i in range(len(w)):
                kyo = Y.iloc[i] - alf
                no = 0
                for j in range(len(model.params)):
                    no += X.iloc[i][j] * model.params[j]
                kyo -= no
                e_i.append(kyo)

            eii = 0
            for i in range(len(w)):
                eii += w[i] * (e_i[i] ** 2)
            delta[q][q] = eii

            factors.loc[q] = [tic, alf, model.params[0], model.params[1], model.params[2], model.pvalues[0],
                              model.pvalues[1],
                              model.pvalues[2], model.rsquared]

        except Exception as e:
            continue
    delta = np.zeros(shape=(len(np.unique(factors["Tic"])), len(np.unique(factors["Tic"]))))
    B = factors[["Beta_mkt_Exc", "Beta_SMB", "Beta_HML"]]
    ## need to rewrite f
    _, idx = np.unique(new_df["SMB value"], return_index=True)

    f = new_df[["datadate", "mkt_excess", "SMB value", "HML value"]].iloc[np.sort(idx)].reset_index(drop=True)
    w = np.zeros(len(f))
    denom = int_sum(len(w))
    for j in range(len(w)):
        w[j] += j + 1
    w = w / denom
    F = np.zeros(shape=(3, 3))
    for t in range(len(f)):

        temp = np.zeros(3)
        for s in range(len(f)):
            temp = temp + w[s] * f.iloc[s][["mkt_excess", "SMB value", "HML value"]]

        term1 = f.iloc[t][["mkt_excess", "SMB value", "HML value"]] - temp

        mat1 = np.dot(np.reshape(np.array(term1), (3, 1)), np.transpose(np.reshape(np.array(term1), (3, 1))))
        mat1 = mat1 * w[t]
        F = F + mat1
    V = np.dot(np.dot(np.array(B), F), np.transpose(np.array(B))) + delta
    V = pd.DataFrame(V)
    delta = pd.DataFrame(delta)
    exp_ret = np.dot(B, f.iloc[-1][1:]) + factors["Alpha"]
    returns = pd.DataFrame()
    returns["tic"] = factors["Tic"]
    returns["expected rets"] = exp_ret
    returns = returns.merge(df[df["datadate"] == xxx][["tic", "chng"]], how="left", on="tic")
    factor_returns = f.iloc[-1][1:]
    load = factors[["Tic", "Alpha", "Beta_mkt_Exc", "Beta_SMB", "Beta_HML"]]
    pvals = factors[["Tic", "p_MX", "p_smb", "p_hml"]]
    V.to_csv(str(data_Path) + "/" + xxx + "_V.txt", sep="\t")
    delta.to_csv(str(data_Path) + "/" + xxx + "_delta.txt", sep="\t")
    returns.to_csv(str(data_Path) + "/" + xxx + "_returns.txt", sep="\t")
    factor_returns.to_csv(str(data_Path) + "/" + xxx + "_factor_returns.txt", sep="\t")
    load.to_csv(str(data_Path) + "/" + xxx + "_loadings_alpha.txt", sep="\t")
    pvals.to_csv(str(data_Path) + "/" + xxx + "_pvals.txt", sep="\t")

/Users/siddharthkantamneni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
